2 CNN: 0.98
1 CNN: 0.9615 epoch 3
1 CNN: 0.9782 epoch 50
ABC: 0.9575 batch 100, lr 0.0001, metrics criterion 'CE', epoch 3
ABC: 0.9671 batch 100, lr 0.0001, metrics criterion 'CE', epoch 6
ABC: 0.9768 epoch 50 kernel_per_pixel 8

In [1]:
import torch
import torchvision
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils.tools import dotdict
from driver.driver import ABC_Driver
from torch_geometric_temporal import METRLADatasetLoader
from other_model.other_model import make_default_model
import atd2022
torch.cuda.is_available()
torch.cuda.set_device(0)

In [2]:
atd_args = dotdict()

atd_args.name = 'atd'
atd_args.train_batch_size = 25
atd_args.predict_len = 4
atd_args.history_len = 7

atd_args.train_epochs= 10
atd_args.lr = 0.00009
atd_args.criterion = 'L1'
atd_args.scheduler = None

activation = 'relu'
pool_name = 'avg'
input_channel = 1
pixel_number = atd_args.history_len*5200
kernel_size = 6
knpp = 12
knpp2 = atd_args.predict_len

atd_args.layers=[
    ('agnostic', ((input_channel, knpp, kernel_size), 1, pool_name, (1,1), activation)),
    ('agnostic', ((knpp, knpp2, kernel_size), 1, pool_name, (1,1), activation)),
    ('linear', (knpp2*atd_args.history_len, atd_args.predict_len, (1,2), 1, (1, atd_args.predict_len))),
]

In [3]:
wt_args = dotdict()

wt_args.name = 'wiki_traffic'
wt_args.train_batch_size = 30
wt_args.predict_len = 1
wt_args.history_len = 5

wt_args.train_epochs= 10
wt_args.lr = 0.001
wt_args.criterion = 'L1'
wt_args.scheduler = None

activation = 'relu'
pool_name = 'avg'
input_channel = 1
pixel_number = wt_args.history_len*1400
kernel_size = 3
knpp = 5
knpp2 = 15

wt_args.layers=[
    ('specific', ((input_channel, knpp, kernel_size), 1, pool_name, (1,1), activation)),
    ('specific', ((knpp, knpp2, kernel_size), 1, pool_name, (1,1), activation)),
    ('linear', (knpp2*wt_args.history_len, wt_args.predict_len, (1,2), 1, (1, wt_args.predict_len))),
]

In [4]:
lat_args = dotdict()

lat_args.name = 'lat'
lat_args.train_batch_size = 30
lat_args.predict_len = 1
lat_args.history_len = 5

lat_args.train_epochs= 10
lat_args.lr = 0.001
lat_args.criterion = 'L1'
lat_args.scheduler = None

activation = 'relu'
pool_name = 'avg'
input_channel = 1
pixel_number = lat_args.history_len*207
kernel_size = 3
knpp = 5
knpp2 = 15

lat_args.layers=[
    ('specific', ((input_channel, knpp, kernel_size), 1, pool_name, (1,1), activation)),
    ('specific', ((knpp, knpp2, kernel_size), 1, pool_name, (1,1), activation)),
    ('linear', (knpp2*lat_args.history_len, lat_args.predict_len, (1,2), 1, (1, lat_args.predict_len))),
]

In [5]:
mnist_args = dotdict()

mnist_args.name = 'mnist'
mnist_args.train_batch_size = 60
mnist_args.predict_batch_size = 100

mnist_args.train_epochs = 10
mnist_args.lr = 0.0001
mnist_args.criterion = 'CE'
mnist_args.scheduler = None

activation = 'relu'
pool_name = 'avg'
input_channel = 1
pixel_number = 28*28
kernel_size = 25
knpp = 10
knpp2 = 20
knpp3 = 39

# mnist_args.layers=[
#     ('specific', ((input_channel, knpp, kernel_size), 1, pool_name, (1,1), activation)),
#     ('specific', ((knpp, knpp2, kernel_size), 1, pool_name, (1,1), activation)),
#     ('specific', ((knpp2, knpp3, kernel_size), 1, pool_name, (1,1), activation)),
#     ('attention', (knpp3, 3)),
#     ('linear', (knpp3, 10, (1,2,3))),
#     ('softmax', (1))
# ]

mnist_args.layers=[
    ('cnn2d', ((input_channel, knpp, (5,5), 1, 2), 1, pool_name, (1,1), activation)),
    ('cnn2d', ((knpp, knpp2, (5,5), 1, 2), 1, pool_name, (1,1), activation)),
    ('cnn2d', ((knpp2, knpp3, (5,5), 1, 2), 1, pool_name, (1,1), activation)),
    ('attention', (knpp3, 3)),
    ('linear', (knpp3, 10, (1,2,3))),
    ('softmax', (1))
]

In [6]:
cifar10_args = dotdict()

cifar10_args.name = 'cifar10'
cifar10_args.train_batch_size = 128
cifar10_args.predict_batch_size = 100

cifar10_args.train_epochs = 180
cifar10_args.lr = 0.1
cifar10_args.criterion = 'CE'
cifar10_args.scheduler = 'multistep'

activation = 'relu'
input_channel = 3
pixel_number = 32*32
kernel_size = 9
# knpp = [63, 63, 129, 129, 255, 255, 513, 513]
knpp = [16, 16, 16, 32, 32, 32, 64, 64, 64]

# cifar10_args.layers=[
#     ('specific', ((input_channel, knpp[0], kernel_size), 2, (1,1), activation)),
#     ('agnostic', ((knpp[0], knpp[1], kernel_size), 2, (2,2), activation)),
#     ('agnostic', ((knpp[1], knpp[2], kernel_size), 2, (1,1), activation)),
#     ('agnostic', ((knpp[2], knpp[3], kernel_size), 2, (2,2), activation)),
#     ('agnostic', ((knpp[3], knpp[4], kernel_size), 2, (1,1), activation)),
#     ('agnostic', ((knpp[4], knpp[5], kernel_size), 2, (2,2), activation)),
#     ('agnostic', ((knpp[5], knpp[6], kernel_size), 2, (1,1), activation)),
#     ('agnostic', ((knpp[6], knpp[7], kernel_size), 2, (2,2), activation)),
# #     ('agnostic', ((knpp[7], knpp[8], kernel_size), 2, (1,1), activation)),
# #     ('agnostic', ((knpp[8], knpp[9], kernel_size), 2, (2,2), activation)),
# #     ('attention', (knpp, 5, 4)),
#     ('linear', (int(knpp[-1]*pixel_number/(4**4)), 10, (1,2,3))),
#     ('softmax', (1))
# ]

# cifar10_args.layers=[
#     ('large', ((input_channel, 33, (3,3), 7, (1,1)), 2, (1,1), activation)),
#     ('large', ((33, 63, (3,3), 7, (1,1)), 2, (1,1), activation)),
#     ('large', ((63, 129, (3,3), 7, (1,1)), 2, (1,1), activation)),
#     ('large', ((129, 255, (3,3), 7, (1,1)), 2, (1,1), activation)),
#     ('linear', (int(33*pixel_number), 10, (1,2,3))),
#     ('softmax', 1)
# ]

cifar10_args.layers=[
    ('cnn2d', ((input_channel, knpp[0], (3,3), 1, 1), 1, (1,1), activation, False)),
    ('cnn2d', ((knpp[0], knpp[0], (3,3), 1, 1), 2, (1,1), activation)),
    ('cnn2d', ((knpp[0], knpp[1], (3,3), 1, 1), 2, (1,1), activation)),
    ('cnn2d', ((knpp[1], knpp[2], (3,3), 1, 1), 2, (1,1), activation)),
    ('cnn2d', ((knpp[2], knpp[3], (3,3), 1, 1), 2, (2,2), activation)),
    ('cnn2d', ((knpp[3], knpp[4], (3,3), 1, 1), 2, (1,1), activation)),
    ('cnn2d', ((knpp[4], knpp[5], (3,3), 1, 1), 2, (1,1), activation)),
    ('cnn2d', ((knpp[5], knpp[6], (3,3), 1, 1), 2, (2,2), activation)),
    ('cnn2d', ((knpp[6], knpp[7], (3,3), 1, 1), 2, (1,1), activation)),
    ('cnn2d', ((knpp[7], knpp[8], (3,3), 1, 1), 2, (1,1), activation)),
    ('adptavgpool', (1,1)),
    ('linear', (knpp[-1], 10, (1,2,3))),
    ('softmax', (1))
]

In [ ]:
args = cifar10_args
data = None
# data = atd2022.io.read_csv()
# data = pd.read_csv('/scratch/mfeng/data/ABC/Wiki_Traffic/filled_selected_train_1.csv', index_col=0, header=[0,1,2,3], parse_dates=True)
# data = pd.read_csv('/scratch/mfeng/data/ABC/LA_Traffic/LA_Traffic.csv', index_col=0)
# data = data.head(200)

driver = ABC_Driver(args, data, record_path=None, if_hash=False)
driver.train()

Use GPU
Files already downloaded and verified
Files already downloaded and verified
add record: 02/01/2023 00:37
epoch: 0, train_loss: 2.1286, test_metric: 0.353
epoch: 1, train_loss: 1.9904, test_metric: 0.4766
epoch: 2, train_loss: 1.9075, test_metric: 0.5112
epoch: 3, train_loss: 1.8593, test_metric: 0.5909
epoch: 4, train_loss: 1.8264, test_metric: 0.5922
epoch: 5, train_loss: 1.7966, test_metric: 0.5472
epoch: 6, train_loss: 1.7764, test_metric: 0.6057
epoch: 7, train_loss: 1.7642, test_metric: 0.5985
epoch: 8, train_loss: 1.7549, test_metric: 0.6787
epoch: 9, train_loss: 1.7459, test_metric: 0.6813
epoch: 10, train_loss: 1.7364, test_metric: 0.6899
epoch: 11, train_loss: 1.7377, test_metric: 0.6275
epoch: 12, train_loss: 1.7271, test_metric: 0.6952
epoch: 13, train_loss: 1.7252, test_metric: 0.6438
epoch: 14, train_loss: 1.7214, test_metric: 0.5518
epoch: 15, train_loss: 1.7223, test_metric: 0.7174
epoch: 16, train_loss: 1.7178, test_metric: 0.6921
epoch: 17, train_loss: 1.7146, 

In [ ]:
driver.metric()

In [8]:
driver.model

ABC_Net(
  (full_modules): ModuleList(
    (0): Conv_Module(
      (first_stage): Sequential(
        (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (mid_stages): ModuleList()
      (last_stage): Sequential()
      (final_activate): ReLU()
    )
    (1): Conv_Module(
      (first_stage): Sequential(
        (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (mid_stages): ModuleList()
      (last_stage): Sequential(
        (0): Sequential(
          (0): ReLU()
          (1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (input_connect): Sequential()
      (final_activate): ReLU()
    )
    (2): C